In [ ]:
#Parallel Cell Algorithm
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Define the grid size
grid_size = 5  # Set a smaller grid for easier display, e.g., 10x10
grid = np.random.randint(0, 2, (grid_size, grid_size))

# Define a simple rule (e.g., Conway's Game of Life)
def game_of_life_cell(i, j, grid):
    # Count live neighbors
    neighbors = grid[i-1:i+2, j-1:j+2]
    total = np.sum(neighbors) - grid[i, j]
    if grid[i, j] == 1:
        return 1 if total in [2, 3] else 0
    else:
        return 1 if total == 3 else 0

def update_cell(i, j, grid):
    return game_of_life_cell(i, j, grid)

def parallel_update(grid):
    # Create a copy of the grid to store updated values
    new_grid = grid.copy()
    with ThreadPoolExecutor() as executor:
        futures = []
        for i in range(1, grid.shape[0]-1):
            for j in range(1, grid.shape[1]-1):
                futures.append(executor.submit(update_cell, i, j, grid))
        # Collect results from the futures and update the new grid
        idx = 0
        for future in futures:
            i, j = divmod(idx, grid.shape[1] - 2)
            new_grid[i+1, j+1] = future.result()
            idx += 1
    return new_grid

# Update the grid
grid = parallel_update(grid)

# Display the grid as a matrix (instead of plotting)
print("Updated Grid after applying Conway's Game of Life rule:")
print(grid)


Updated Grid after applying Conway's Game of Life rule:
[[1 1 0 0 1]
 [0 0 1 0 0]
 [1 0 0 0 1]
 [0 0 0 0 1]
 [0 1 1 1 1]]


In [ ]:
import random
import math
import copy

# Fitness functions
def fitness_rastrigin(position):
    return sum(x**2 - 10 * math.cos(2 * math.pi * x) + 10 for x in position)

def fitness_sphere(position):
    return sum(x**2 for x in position)

# Wolf class
class Wolf:
    def __init__(self, fitness, dim, minx, maxx, seed):
        self.rnd = random.Random(seed)
        self.position = [self.rnd.uniform(minx, maxx) for _ in range(dim)]
        self.fitness = fitness(self.position)

def gwo(fitness, max_iter, n, dim, minx, maxx):
    rnd = random.Random(0)
    population = [Wolf(fitness, dim, minx, maxx, i) for i in range(n)]
    population.sort(key=lambda w: w.fitness)

    alpha_wolf, beta_wolf, gamma_wolf = copy.deepcopy(population[:3])

    for Iter in range(max_iter):
        if Iter % 10 == 0 and Iter > 0:
            print(f"Iter = {Iter} best fitness = {alpha_wolf.fitness:.3f}")

        a = 2 * (1 - Iter / max_iter)

        for wolf in population:
            A1, A2, A3 = [a * (2 * rnd.random() - 1) for _ in range(3)]
            C1, C2, C3 = [2 * rnd.random() for _ in range(3)]

            X1 = [alpha_wolf.position[j] - A1 * abs(C1 * alpha_wolf.position[j] - wolf.position[j]) for j in range(dim)]
            X2 = [beta_wolf.position[j] - A2 * abs(C2 * beta_wolf.position[j] - wolf.position[j]) for j in range(dim)]
            X3 = [gamma_wolf.position[j] - A3 * abs(C3 * gamma_wolf.position[j] - wolf.position[j]) for j in range(dim)]

            Xnew = [(X1[j] + X2[j] + X3[j]) / 3.0 for j in range(dim)]
            fnew = fitness(Xnew)

            if fnew < wolf.fitness:
                wolf.position = Xnew
                wolf.fitness = fnew

        population.sort(key=lambda w: w.fitness)
        alpha_wolf, beta_wolf, gamma_wolf = copy.deepcopy(population[:3])

    return alpha_wolf.position

# Driver code

def run_gwo(fitness, func_name, dim=3, num_particles=50, max_iter=100, minx=-10.0, maxx=10.0):
    print(f"\nBegin grey wolf optimization on {func_name} function\n")
    print(f"Goal is to minimize {func_name} in {dim} variables")
    print(f"Function has known min = 0.0 at ({', '.join(['0'] * dim)})")
    print(f"Setting num_particles = {num_particles}")
    print(f"Setting max_iter = {max_iter}\n")

    best_position = gwo(fitness, max_iter, num_particles, dim, minx, maxx)
    print(f"\nGWO completed\n")
    print(f"Best solution found: {[f'{x:.6f}' for x in best_position]}")
    print(f"Fitness of best solution = {fitness(best_position):.6f}\n")

# Run GWO for Rastrigin and Sphere functions
run_gwo(fitness_rastrigin, "Rastrigin")
run_gwo(fitness_sphere, "Sphere")



Begin grey wolf optimization on Rastrigin function

Goal is to minimize Rastrigin in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter = 100

Iter = 10 best fitness = 2.019
Iter = 20 best fitness = 1.004
Iter = 30 best fitness = 0.996
Iter = 40 best fitness = 0.996
Iter = 50 best fitness = 0.995
Iter = 60 best fitness = 0.995
Iter = 70 best fitness = 0.995
Iter = 80 best fitness = 0.995
Iter = 90 best fitness = 0.995

GWO completed

Best solution found: ['-0.994930', '-0.000258', '0.001483']
Fitness of best solution = 0.995409


Begin grey wolf optimization on Sphere function

Goal is to minimize Sphere in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter = 100

Iter = 10 best fitness = 0.000
Iter = 20 best fitness = 0.000
Iter = 30 best fitness = 0.000
Iter = 40 best fitness = 0.000
Iter = 50 best fitness = 0.000
Iter = 60 best fitness = 0.000
Iter = 70 best fitness = 0.000
Iter = 80 best 

In [ ]:
# Cuckoo Search Algorithm

import random
import networkx as nx
import numpy as np
import math
import datetime
import pandas as pd

class Cuckoo:
    def __init__(self, path, G, eps = 0.9):
        self.path = path
        self.G = G
        self.nodes = list(G.nodes)
        self.eps = eps
        self.fitness = self.calculate_fitness()

    """
    Function to Compute fitness value.
    """
    def calculate_fitness(self):
        fitness = 0.0

        for i in range(1, len(self.path)):
            total_distance = 0
            curr_node = self.path[i-1]
            next_node = self.path[i]
            if self.G.has_edge(curr_node, next_node):
                fitness += self.G[curr_node][next_node]['weight']
            else:
                fitness += 0
        fitness = np.power(abs(fitness + self.eps), 2)
        return fitness

    def generate_new_path(self):
        """
        This function generates a random solution (a random path) in the graph
        """
        nodes = list(self.G.nodes)
        start = nodes[0]
        end = nodes[-1]
        samples = list(nx.all_simple_paths(self.G, start, end))
        for i in range(len(samples)):
            if len(samples[i]) != len(nodes):
                extra_nodes = [node for node in nodes if node not in samples[i]]
                random.shuffle(extra_nodes)
                samples[i] = samples[i] + extra_nodes

        sample_node = random.choice(samples)
        return sample_node

class CuckooSearch:
    def __init__(self, G, num_cuckoos, max_iterations, beta):
        self.G = G
        self.nodes = list(G.nodes)
        self.num_cuckoos = num_cuckoos
        self.max_iterations = max_iterations
        self.beta = beta
        self.cuckoos = [Cuckoo(random.sample(self.nodes, len(self.nodes)), self.G) for _ in range(self.num_cuckoos)]
        self.test_results = []
        self.test_cases = 0

    """
    Function to buld new nests at new location and abandon old ones using Levi flights.
    """
    def levy_flight(self):
        sigma = (math.gamma(1 + self.beta) * np.sin(np.pi * self.beta / 2) / (math.gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))) ** (1 / self.beta)
        u = np.random.normal(0, sigma, 1)
        v = np.random.normal(0, 1, 1)
        step = u / (abs(v) ** (1 / self.beta))
        return step

    def optimize(self):
        for i in range(self.max_iterations):
            for j in range(self.num_cuckoos):
                cuckoo = self.cuckoos[j]
                step = self.levy_flight()
                new_path = cuckoo.generate_new_path()
                new_cuckoo = Cuckoo(new_path, self.G)
                if new_cuckoo.fitness > cuckoo.fitness:
                    self.cuckoos[j] = new_cuckoo
                    self.test_cases+=1

            self.cuckoos = sorted(self.cuckoos, key=lambda x: x.fitness, reverse=True)
            best_path=self.cuckoos[0].path
            best_fitness=self.cuckoos[0].fitness

            self.test_results.append([i, best_fitness, self.test_cases])

        last_node = list(self.G.nodes)[-1]
        last_node_index = best_path.index(last_node) + 1

        return best_path[:last_node_index], best_fitness

if __name__ == "__main__":
    """
    Example usage
    """
    Gn = nx.DiGraph()

    #Add nodes to the graph
    for i in range(11):
        Gn.add_node(i)

    edges = [(0, 1,{'weight': 1}), (1, 3,{'weight': 2}), (1, 2,{'weight': 1}),(2, 4,{'weight': 2}),
            (3, 2,{'weight': 2}),(3, 4,{'weight': 1}),(3, 5,{'weight': 2}),(3, 7,{'weight': 4}),
            (4, 5,{'weight': 1}),(4, 6,{'weight': 2}),(5, 7,{'weight': 2}),(5, 8,{'weight': 3}),
            (6, 7,{'weight': 1}),(7, 9,{'weight': 2}),(8, 10,{'weight': 2}),(9, 10,{'weight': 1})]

    Gn.add_edges_from(edges)

    csa = CuckooSearch(Gn, num_cuckoos = 30, max_iterations=1000, beta=0.27)

    start = datetime.datetime.now()
    best_path, best_fitness = csa.optimize()
    end = datetime.datetime.now()

    csa_time = end - start

    csa_test_data = pd.DataFrame(csa.test_results,columns = ["iterations","fitness_value","test_cases"])

    print("Optimal path: ", best_path)
    print("Optimal path cost: ", best_fitness)
    print("CSA total Exec time => ", csa_time.total_seconds())
    csa_test_data.to_csv("csa_test_data_results.csv")

Optimal path:  [0, 1, 3, 7, 9, 10]
Optimal path cost:  320.40999999999997
CSA total Exec time =>  16.442699


In [ ]:
from __future__ import division
import random
import math

#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)
def func1(x):
    total = 0
    for i in range(len(x)):
        total += x[i]**2
    return total

#--- PARTICLE CLASS -----------------------------------------------------------+

class Particle:
    def __init__(self, x0, bounds):
        self.position_i = []          # particle position
        self.velocity_i = []          # particle velocity
        self.pos_best_i = []          # best position individual
        self.err_best_i = float('inf')  # best error individual
        self.err_i = float('inf')       # error individual

        for i in range(len(x0)):
            self.velocity_i.append(random.uniform(-1, 1))
            self.position_i.append(random.uniform(bounds[i][0], bounds[i][1]))

    # evaluate current fitness
    def evaluate(self, costFunc):
        self.err_i = costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i:
            self.pos_best_i = self.position_i[:]
            self.err_best_i = self.err_i

    # update new particle velocity
    def update_velocity(self, pos_best_g, w=0.5, c1=1.5, c2=2.0):
        for i in range(len(self.position_i)):
            r1 = random.random()
            r2 = random.random()

            vel_cognitive = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            vel_social = c2 * r2 * (pos_best_g[i] - self.position_i[i])
            self.velocity_i[i] = w * self.velocity_i[i] + vel_cognitive + vel_social

    # update the particle position based off new velocity updates
    def update_position(self, bounds):
        for i in range(len(self.position_i)):
            self.position_i[i] += self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            # adjust minimum position if necessary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]

#--- PSO CLASS ----------------------------------------------------------------+

class PSO:
    def __init__(self, costFunc, x0, bounds, num_particles, maxiter):
        self.num_dimensions = len(x0)
        self.err_best_g = float('inf')       # best error for group
        self.pos_best_g = []                 # best position for group
        self.swarm = [Particle(x0, bounds) for _ in range(num_particles)]
        self.costFunc = costFunc
        self.bounds = bounds
        self.num_particles = num_particles
        self.maxiter = maxiter

    def optimize(self):
        for i in range(self.maxiter):
            for particle in self.swarm:
                particle.evaluate(self.costFunc)

                # determine if current particle is the best (globally)
                if particle.err_i < self.err_best_g:
                    self.pos_best_g = particle.position_i[:]
                    self.err_best_g = particle.err_i

            for particle in self.swarm:
                particle.update_velocity(self.pos_best_g)
                particle.update_position(self.bounds)

            # Print iteration details
            print(f"Iteration {i+1}/{self.maxiter}, Best Fitness: {self.err_best_g}")

        # Print final results
        print('FINAL RESULTS:')
        print(f"Best Position: {self.pos_best_g}")
        print(f"Best Fitness: {self.err_best_g}")

#--- RUN ----------------------------------------------------------------------+

if __name__ == "__main__":
    initial = [5, 5]               # initial starting location [x1, x2, ...]
    bounds = [(-10, 10), (-10, 10)]  # input bounds [(x1_min, x1_max), (x2_min, x2_max), ...]
    num_particles = 15
    maxiter = 30

    print("Starting Particle Swarm Optimization...")
    optimizer = PSO(func1, initial, bounds, num_particles, maxiter)
    optimizer.optimize()


Starting Particle Swarm Optimization...
Iteration 1/30, Best Fitness: 1.3261652700490267
Iteration 2/30, Best Fitness: 0.6674217263090517
Iteration 3/30, Best Fitness: 0.1455954928741141
Iteration 4/30, Best Fitness: 0.0052876812915318636
Iteration 5/30, Best Fitness: 0.0052876812915318636
Iteration 6/30, Best Fitness: 0.0052876812915318636
Iteration 7/30, Best Fitness: 0.0052876812915318636
Iteration 8/30, Best Fitness: 0.004988617240709642
Iteration 9/30, Best Fitness: 0.004988617240709642
Iteration 10/30, Best Fitness: 0.004988617240709642
Iteration 11/30, Best Fitness: 0.004988617240709642
Iteration 12/30, Best Fitness: 0.003163958183691373
Iteration 13/30, Best Fitness: 0.002491159140649002
Iteration 14/30, Best Fitness: 0.002491159140649002
Iteration 15/30, Best Fitness: 0.0003614632318451565
Iteration 16/30, Best Fitness: 0.0003614632318451565
Iteration 17/30, Best Fitness: 0.0003614632318451565
Iteration 18/30, Best Fitness: 0.0003614632318451565
Iteration 19/30, Best Fitness: 

In [ ]:
# Ant Colony Optimization

import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iterations (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): Exponent on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): Exponent on distance, higher beta give distance more weight. Default=1
        """
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print(f"Iteration {i + 1}, shortest path: {shortest_path}")
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone = self.pheromone * self.decay
        return all_time_shortest_path

    def spread_pheromone(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # going back to where we started
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move

# Example usage
if __name__ == "__main__":
    distances = np.array([[np.inf, 2, 2, 5, 7],
                          [2, np.inf, 4, 8, 2],
                          [2, 4, np.inf, 1, 3],
                          [5, 8, 1, np.inf, 2],
                          [7, 2, 3, 2, np.inf]])

    ant_colony = AntColony(distances, n_ants=5, n_best=1, n_iterations=10, decay=0.95, alpha=1, beta=1)
    shortest_path = ant_colony.run()
    print("Final shortest path:", shortest_path)


Iteration 1, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 2, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 3, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 4, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 5, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 6, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 7, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 8, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 9, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Iteration 10, shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)
Final shortest path: ([(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)], 9.0)


In [ ]:
import random
import numpy as np

# Fitness function
def fitness(chromosome):
    x = int(''.join(map(str, chromosome)), 2)
    return x ** 2

# Generate a random chromosome
def generate_chromosome(length):
    return [random.randint(0, 1) for _ in range(length)]

# Generate an initial population
def generate_population(size, chromosome_length):
    return [generate_chromosome(chromosome_length) for _ in range(size)]

# Select two parents based on their fitness
def select_pair(population, fitnesses):
    total_fitness = sum(fitnesses)
    selection_probs = [f / total_fitness for f in fitnesses]
    parent1 = population[random.choices(range(len(population)), weights=selection_probs)[0]]
    parent2 = population[random.choices(range(len(population)), weights=selection_probs)[0]]
    return parent1, parent2

# Crossover between two parents
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:point] + parent2[point:]
    offspring2 = parent2[:point] + parent1[point:]
    return offspring1, offspring2

# Mutate a chromosome
def mutate(chromosome, mutation_rate):
    return [gene if random.random() > mutation_rate else 1 - gene for gene in chromosome]

# Parameters
population_size = 10
chromosome_length = 5
mutation_rate = 0.01
generations = 20

# Initialize population
population = generate_population(population_size, chromosome_length)

for generation in range(generations):
    # Calculate fitness for each chromosome
    fitnesses = [fitness(chromosome) for chromosome in population]
    print(f"Generation {generation + 1} best fitness: {max(fitnesses)}")

    # Create a new generation
    new_population = []
    for _ in range(population_size // 2):
        parent1, parent2 = select_pair(population, fitnesses)
        offspring1, offspring2 = crossover(parent1, parent2)
        offspring1 = mutate(offspring1, mutation_rate)
        offspring2 = mutate(offspring2, mutation_rate)
        new_population.extend([offspring1, offspring2])

    # Replace the old population
    population = new_population



Generation 1 best fitness: 900
Generation 2 best fitness: 900
Generation 3 best fitness: 900
Generation 4 best fitness: 900
Generation 5 best fitness: 961
Generation 6 best fitness: 961
Generation 7 best fitness: 961
Generation 8 best fitness: 961
Generation 9 best fitness: 961
Generation 10 best fitness: 961
Generation 11 best fitness: 961
Generation 12 best fitness: 961
Generation 13 best fitness: 961
Generation 14 best fitness: 961
Generation 15 best fitness: 961
Generation 16 best fitness: 961
Generation 17 best fitness: 961
Generation 18 best fitness: 961
Generation 19 best fitness: 961
Generation 20 best fitness: 961
